In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from time import sleep
from selenium.webdriver.common.by import By

import http.client  # or http.client if you're on Python 3
http.client._MAXHEADERS = 10000

import warnings 
warnings.filterwarnings(action= 'ignore')

# Test

In [8]:
from selenium import webdriver

main_link = "https://www.contactcars.com/en/cars/used/engines?page=1&sortOrder=false&sortBy=CreatedAt"

# chrome = webdriver.Chrome(executable_path="/home/haytham/Downloads/chromedriver_linux64/chromedriver")

chrome = webdriver.Firefox()

In [9]:
def get_links_list(link, driver):
    
    driver.get(link)
    scroll_down(driver)
#     z = chrome.find_elements_by_css_selector("d-block bg--white car-link")
    z = driver.find_elements(by= By.CLASS_NAME,value="car-link")

    lnk_list = []

    for i in z:
#         print(i.get_attribute("href"))
        lnk_list.append(i.get_attribute("href"))

    return lnk_list



def scroll_down(driver):
    import time
    SCROLL_PAUSE_TIME = 1.7

# Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height



def get_ad_details(l):

    car_dict = {}
# l = "https://www.contactcars.com/en/car/626997664f4dc4c0a1d6a69f-Chevrolet-Optra-1.5-A-T-H-L?status=Used"
    chrome.get(l)
    chrome.implicitly_wait(1)


    # car_details var list of WEBELEMENT include 
#     ['SUV','Body Shape','Automatic','Transmission','3,500 KM'
# ,'Mileage','Green','Color','1200 CC','Engine Capacity','Yes','Fabrica?']

    car_details = chrome.find_elements(by= By.CLASS_NAME,value="detail-cards")[-1].text.split("\n")
#     print(car_details)
    # car_info var like ['Opel', 'Mokka', '2022', 'Used', 'Egypt', '9 days ago', '855,000 EGP']
    car_info = chrome.find_elements(by= By.CLASS_NAME,value="used-car")[-1].text.split("\n")
#     print(car_info)

    
    print(l.split("car/")[-1].split("-")[0])
    car_dict["Ad_ID"]= l.split("car/")[-1].split("-")[0]
    
    car_dict.update(extract_details(car_details))
    
#     car_dict["Body Type"] = car_details[0]
#     car_dict["Engine Capacity (CC)"] = car_details[8]
#     car_dict["Transmission Type"] = car_details[2]
#     car_dict["Kilometers"] = car_details[4]
#     car_dict["Factory Paint"] = car_details[10]

    car_dict["Condition"] = car_info[3]
    car_dict["Year"] = car_info[2]
    car_dict["brand"] = car_info[0]
    car_dict["Model"] = car_info[1]
    car_dict["date"] =  car_info[-2]
    car_dict["price"] = car_info[-1]
    
    print(car_dict)

    return car_dict


def get_page_link(nu):
    ma ="https://www.contactcars.com/en/cars/used/engines?page="
    ink = "&sortOrder=false&sortBy=CreatedAt"
    return ma+ str(nu) + ink

def get_brand_model(string):

    import math
    s = string.split(" ")[0:-1]

    model = " ".join( s[math.floor(len(s)/2) :])
    brand = " ".join( s[: math.floor(len(s)/2)])
#     print(brand, "\n", model,"\n_______")
    return brand, model

def isFabrika(webElementList):
    fabrika = False
    for i in webElementList:
        if i.text == 'Factory Paint':
#             print("Fabrika")
            fabrika= True
        else:
            fabrika= False
    return fabrika


def extract_details(mlist):

    mdic = {}
    i = 0
    while i < len(mlist):
        if (i % 2) == 0:
            mdic[mlist[i+1]] = mlist[i]
        i+=1
        
    return mdic

In [10]:
df = pd.DataFrame()

for i in range(1,150):

    links_list =  get_links_list(get_page_link(i), chrome)

    try:
        
        for link in links_list:
            
            try:
                print("____________________")
                print(link)

                car_dict = get_ad_details(link)

                df = df.append(car_dict, ignore_index = True)
            except Exception as e2:
                print("inside exption")
#                 print(car_dict)
                print(e2)
                continue
    
    except Exception as e:
        print("outer excption")
        print(e)
        continue
        
    
    print(df.tail(1))
    
print("End Of Data Scrape")

____________________
https://www.contactcars.com/en/car/9551c6b0ece54461ae937d56a89be50e-Audi-A5-2.0-A-T-Sportback-S-Line%09?status=Used
9551c6b0ece54461ae937d56a89be50e
{'Ad_ID': '9551c6b0ece54461ae937d56a89be50e', 'Body Shape': 'Sedan', 'Transmission': 'Automatic', 'Mileage': '100 KM', 'Engine Capacity': '2000 CC', 'Condition': 'Used', 'Year': '2023', 'brand': 'Audi', 'Model': 'A5', 'date': '3 days ago', 'price': '3,225,000 EGP'}
____________________
https://www.contactcars.com/en/car/dde1829d1e004b49a9d36f4fe35eef4a-Chevrolet-Optra-1.5-A-T-H-L?status=Used
dde1829d1e004b49a9d36f4fe35eef4a
{'Ad_ID': 'dde1829d1e004b49a9d36f4fe35eef4a', 'Body Shape': 'Sedan', 'Transmission': 'Automatic', 'Mileage': '38 KM', 'Engine Capacity': '1500 CC', 'Condition': 'Used', 'Year': '2023', 'brand': 'Chevrolet', 'Model': 'Optra', 'date': '5 days ago', 'price': '450,000 EGP'}
____________________
https://www.contactcars.com/en/car/2fa7a9ecac41479a99e3ec4a52910bc3-Kia-Sportage-1.6-A-T-LX-F-O-?status=Used
2

73a3b248dd5f45f7a07a52db2e4b1e08
{'Ad_ID': '73a3b248dd5f45f7a07a52db2e4b1e08', 'Body Shape': 'SUV', 'Transmission': 'Automatic', 'Mileage': '22,000 KM', 'Engine Capacity': '4400 CC', 'Condition': 'Used', 'Year': '2021', 'brand': 'BMW', 'Model': 'X6', 'date': 'about 3 hours ago', 'price': '5,000,000 EGP'}
____________________
https://www.contactcars.com/en/car/38cb2751e91a410798722386cb18f0e4-Mercedes-E-180-1.6-A-T-Premium?status=Used
38cb2751e91a410798722386cb18f0e4
{'Ad_ID': '38cb2751e91a410798722386cb18f0e4', 'Body Shape': 'Sedan', 'Transmission': 'Automatic', 'Mileage': '1,000 KM', 'Engine Capacity': '1600 CC', 'Condition': 'Used', 'Year': '2019', 'brand': 'Mercedes', 'Model': 'E 180', 'date': 'about 3 hours ago', 'price': '2,150,000 EGP'}
____________________
https://www.contactcars.com/en/car/0a1c37e18dac453e81f14abf7fef3e86-Dodge-Ram?status=Used
0a1c37e18dac453e81f14abf7fef3e86
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/ca1efb36

24aae0e5dd6d49eaaef89d53e5f0a5c7
{'Ad_ID': '24aae0e5dd6d49eaaef89d53e5f0a5c7', 'Body Shape': 'Sedan', 'Transmission': 'Automatic', 'Mileage': '111,000 KM', 'Engine Capacity': '3000 CC', 'Condition': 'Used', 'Year': '2015', 'brand': 'BMW', 'Model': '535i', 'date': 'about 3 hours ago', 'price': '1,250,000 EGP'}
____________________
https://www.contactcars.com/en/car/eb823a70945348a09d8654972657e94b-Mercedes-A180-1.3-A-T-Premium-Metallic-?status=Used
eb823a70945348a09d8654972657e94b
{'Ad_ID': 'eb823a70945348a09d8654972657e94b', 'Body Shape': 'Sedan', 'Transmission': 'Automatic', 'Mileage': '2,500 KM', 'Color': 'Gray', 'Engine Capacity': '1300 CC', 'Fabrica?': 'Yes', 'Condition': 'Used', 'Year': '2021', 'brand': 'Mercedes', 'Model': 'A180', 'date': 'about 3 hours ago', 'price': '1,450,000 EGP'}
____________________
https://www.contactcars.com/en/car/3c18d14e889843398d98da8609ad113e-MG-ZS-1.5-A-T-Comfort?status=Used
3c18d14e889843398d98da8609ad113e
{'Ad_ID': '3c18d14e889843398d98da8609ad113

inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/4c849f73aaee43858bf640ca48161609-Renault-Fluence?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/e7bdf5c905f04d2cb02e2435f9ebfa10-Mercedes-E200?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/3421182056c54a009199e8333a68b81f-Suzuki-Ciaz-1.5-A-T-GLX?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/df9d8a15b6934651b28a3d6c36c74cfc-Hyundai-Elantra-CN7-1.6-A-T-CN7-Premium-?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/7dab8345766e4c6fa70fa6688f59c4f8-Chevrolet-Optra-1.5-A-T-H-L?status=Used
7dab8345766e4c6fa70fa6688f59c4f8
{'Ad_ID': '7dab8345766e4c6fa70fa6688f59c4f8', 'Body Shape': 'Sedan', 'Transmission': 'Automatic', 'Mileage': '53 KM', 'Engine Capacity': '1500 C

inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/b1d0c91ccba0412db2bf53dd0625a696-Audi-A5-2.0-A-T-Sportback-S-Line%09?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/18e7329ea930452da7d98b42eaa3278d-Subaru-XV-1.6-A-T-H-L?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/9136b1057fe446f6af87d5584660f904-Chery-Envy?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/3ef763a47e9b485baf7424b2677e5a18-Tesla-Model--Y?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/63b56ae4fb2856877a64b2fb-Hyundai-Tucson-1.6T-GDI-Topline?status=Used
inside exption
list index out of range
____________________
https://www.contactcars.com/en/car/63b56acc574a0a80361d3bff-Hyundai-Tucson-1.6-A-T-GLS-Plus-TLe-FL?status=Used
inside exption
list i

inside exption
Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.contactcars.com/en/car/de7bf2c018d246f283571771ca05d728-Mercedes-GLA-200-1.3-A-T-PROGRESSIVE-%3Fstatus%3DUsed&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20www.contactcars.com.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:61:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:329:39
emit@resource://gre/modules/EventEmitter.sys.mjs:154:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:35:25

____________________
https://www.contactcars.com/en/car/7c9e5c72a3c2461fa838d0d84601d700-Mercedes-A200?status=Used
inside exption
Message: Reached error page: about:neterror?e=dnsNotFoun

inside exption
Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.contactcars.com/en/car/cd6a7fecab2a4d9cb327173cc00be0d9-Opel-Astra%3Fstatus%3DUsed&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20www.contactcars.com.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:61:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:329:39
emit@resource://gre/modules/EventEmitter.sys.mjs:154:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:35:25

____________________
https://www.contactcars.com/en/car/9a36bea3732843e1983a951bc08ec4a5-Lada-2105?status=Used
inside exption
Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.contactcars.c

inside exption
Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.contactcars.com/en/car/3fc1ec77732b4ecba30274a2ba1b70d8-Toyota-Fortuner-4.0-A-T-F-O-4WD-FL%3Fstatus%3DUsed&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20www.contactcars.com.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:61:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:329:39
emit@resource://gre/modules/EventEmitter.sys.mjs:154:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:35:25

____________________
https://www.contactcars.com/en/car/6840f57b2a1749cea1fc0ff9848f4d39-Mercedes-E300e-Amg-2.0-A-T-e-AMG?status=Used
inside exption
Message: Reached error page: about:neter

inside exption
Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.contactcars.com/en/car/f3e348d8b5984c09a0e790c39ab21592-Nissan-Motor-Egypt-Qashqai-1.2-A-T-Sport%3Fstatus%3DUsed&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20www.contactcars.com.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:61:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:329:39
emit@resource://gre/modules/EventEmitter.sys.mjs:154:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:35:25

____________________
https://www.contactcars.com/en/car/de1f796e20d4498996db17bb5ed8439f-Hyundai-Elantra-HD-1.6-GL-A-T?status=Used
inside exption
Message: Reached error page: about:ne

WebDriverException: Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.contactcars.com/en/cars/used/engines%3Fpage%3D3%26sortOrder%3Dfalse%26sortBy%3DCreatedAt&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20www.contactcars.com.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:61:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:329:39
emit@resource://gre/modules/EventEmitter.sys.mjs:154:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:35:25


In [ ]:
df

In [ ]:
df = df[["Ad_ID","date","Body Type","Condition", "Engine Capacity (CC)","Kilometers","Model", "Transmission Type","Year","brand","price"]]
df[100:250]

In [ ]:
df.to_csv("contact_cars2.csv")
df.shape

In [ ]:
chrome.get("https://www.contactcars.com/en/car/6283a64006683e8b604f2af1-Renault-Logan-Basic?status=Used")

In [ ]:
car_details = chrome.find_elements_by_class_name("car-details__sumary__item__details")


In [ ]:
car_details[2].text

In [ ]:
df["date"].unique()

In [ ]:
df = df.drop(df[df["date"].str.contains(pat= "Zayed", na=False, regex= True)].index)

In [ ]:
chrome.quit